In [1]:
import numpy as np

# An Example in Deriving Values for WFE, WFS&C Factors, and Sensitivities  
This write-up's intent is to document a preliminary example following the guidelines below:
- State the underlying assumption, rationale, or reference for each assumed value
- Maintain internal consistency
    * Relationships between different values should be consistent with underlying rationales and physical principles
    * Avoid double counting

## Assumed spatial-frequency, temporal-frequency and angular-separation bins  
###  Temporal-Frequency Bins (row labels in WFE and WFS&C arrays)  
#### Based on USORT Bins (L. Coyle, B. Sitarsky).  Ref:  slide # 6 in "2023.03.31_ULTRA_Stability_Budgets.pptx" by L. Coyle
* Static:  0 Hz  
* LF1:  5.8$\times$10$^{-6}$ - 0.001 Hz
    - 48 hr - 1000 s.  Observing scenario and coronagraph HOWFS bandwidth, depending on target-star brightness
* LF2:  0.001 - 0.01 Hz
    - Coronagraph LOWFS bandwidth, depending on target-star brightness  
* LF3:  0.01 - 1 Hz
    - Telescope alignment (PM/SM rigid body motion) bandwidth  
* MF:  1 - 10 Hz  
    - PM segment-level, rigid-body sensing & control  
* HF:  > 10 Hz  
    - Uncontrolled or effects removed w/ image processing  
    
### Spatial-Frequency Bins (column labels in WFE, WFS&C factor, and sensitivity arrays)  
#### Based on USORT Bins (L. Coyle, B. Sitarsky).  Ref:  slide # 6 in "2023.03.31_ULTRA_Stability_Budgets.pptx" by L. Coyle
* Low:  Noll Z2 - Z11
    - Global alignment of PM; low order PM modes (backplane); Can be compensated w/ SM motion  
* Mid:  4 - 15 cyc/dia  
    - PMSA rigid body motion; low order PMSA modes  
* High:  15 - 60 cyc/dia  
    - PMSA mid spatial modes (i.e. mount print through)  
* High+:  > 60 cyc/dia  
    - PMSA high spatial modes above the DM correction range (outside dark hole but considering aliasing into the science field)  

### Angular-Separation Bins (row labels in sensitivity arrays)  
* HZs of fiducial stars span 53.5 - 124.0 mas angular range, from inner to outer edges.  
* For D=6.52 m (USORT pupil incribed diameter), 
    - 6.8 - 15.7 $\lambda$/D @250nm, 
    - 3.1 - 7.1 $\lambda$/D @550nm, 
    - 1.0 - 2.3 $\lambda$/D @1.7$\mu$m  
* For vis, let's use the following bins:  
    1. 35 - 52 mas (2 - 3 $\lambda/D$)  
    2. 52 - 70 mas (3 - 4 $\lambda/D$)  
    3. 70 - 87 mas (4 - 5 $\lambda/D$)  
    4. 87 - 104 mas (5 - 6 $\lambda/D$)    
    5. 104 - 122 mas (6 - 7 $\lambda/D$)  
    6. 122 - 139 mas (6 - 8 $\lambda/D$)  

In [2]:
print(f"HZ inner to outer edge @ 250 nm:  {(53.5e-3/(206266*250e-9/6.52)):.1f} -  {(124e-3/(206266*250e-9/6.52)):.1f}")
print(f"HZ inner to outer edge @ 550 nm:  {(53.5e-3/(206266*550e-9/6.52)):.1f} -  {(124e-3/(206266*550e-9/6.52)):.1f}")
print(f"HZ inner to outer edge @ 1700 nm:  {(53.5e-3/(206266*1700e-9/6.52)):.1f} -  {(124e-3/(206266*1700e-9/6.52)):.1f}")

HZ inner to outer edge @ 250 nm:  6.8 -  15.7
HZ inner to outer edge @ 550 nm:  3.1 -  7.1
HZ inner to outer edge @ 1700 nm:  1.0 -  2.3


In [3]:
for i in range(2, 9):
    print(f"{1e3*206266*i*550e-9/6.52:.0f} mas @ {i} lambda/D")

35 mas @ 2 lambda/D
52 mas @ 3 lambda/D
70 mas @ 4 lambda/D
87 mas @ 5 lambda/D
104 mas @ 6 lambda/D
122 mas @ 7 lambda/D
139 mas @ 8 lambda/D


## WFE  
Based on slide # 10 in "2023.03.31_ULTRA_Stability_Budgets.pptx" by L. Coyle    

|      |Low|Mid|High|High+|  
|------|---|---|----|-----|  
|Static|194029|174.4|0.2|0|  
|LF1   |194029|174.4|0.2|0|    
|LF2   |194029|34.7|0.2|0|
|LF3   |19403|22.6|0.2|0|  
|MF    |9701|22.6|0.2|0|
|HF    |1940|12.2|0.2|0|  


## WFSC  
### Account for only LOWFS here.  HOWFS or dark-hole digging is folded into sensitivities.  
- In the "low" spatial frequency column:  
    - "Static" value
        - Just a guess  
    - "LF1," "LF2" and "LF3" values 
        - Fig. 6(a), g = 0.2 curve, in Potier et al. 2022  
        - The curve is approximately $|H|^2 = \left(\frac{f/f_s}{0.03}\right)^2$, the sampling frequency was 100 Hz.    
    - Therefore, average over a frequency interval is $|H|_{avg} = \sqrt{\frac{370\left[(f_2/f_s)^3 - (f_1/f_s)^3\right]}{(f_2/f_s) - (f_1/f_s)}}$    
- Other columns:  assume no suppression 

|      |Low|Mid|High|High+|  
|------|---|---|----|-----|  
|Static|3$\times$10$^{-6}$|1|1|1|  
|LF1   |2$\times$10$^{-4}$|1|1|1|    
|LF2   |2$\times$10$^{-3}$|1|1|1|
|LF3   |0.2|1|1|1|  
|MF    |1|1|1|1|
|HF    |1|1|1|1|  
    

In [4]:
def wfsc(f1, f2, fs=100):
    return np.sqrt(370*((f2/fs)**3 - (f1/fs)**3)/((f2/fs - f1/fs)))

In [5]:
print(wfsc(5.8e-6, 0.001))
print(wfsc(0.001, 0.01))
print(wfsc(0.01, 1))

0.00019291408626639995
0.002026573462769115
0.19332278706867434


# Sensitivity  
- For "low" spatial frequency
    * For the first 2 angular-separation bins  
        * Fig. 26(b) in Nemati et al. 2020  
        * Use the RSS allocation (40 ppt) divided by the RSS tolerances (6.21 pm) $\frac{40}{6.21} = 6.4$ ppt/pm 
- For "mid" spatial-frequency bins  
    * Eyeball the Lowest curve in "Piston/tip/tilt" panel in lower row of Fig. 12 in Juanola-Parramon et al. 2022 (for VVC LUVOIR-B), which corresponds to contrast change cause by 2.9 pm RMS WFE.  
        - 10 ppt/pm @ 2 & 3 $\lambda$/D  
        - 3 ppt/pm @ 4 & 5 $\lambda$/D  
        - 1.5 ppt/pm @ 6 - 8 $\lambda$/D  
    * Mainly affect angular bins 3 - 6 
- For "high" spatial-frequency bin:  Above the considered angular range, although can have frequency-mixing effects.  Assume null values for now.  
- For "high+" spatial-frequency bin.  No speckle suppression because the spatial frequencies are above DM correction range. 
    * For each mode, $I_{speckle} = \left(\frac{\pi h}{\lambda}\right)^2$   
    * Differentiate:  $\frac{dI}{dh} = \frac{2\pi}{\lambda}\sqrt{I}$. This yields 114 ppt/pm for $I=1\times10^{-10}$.
    * The spatial frequencies are far above the considered angular range, but can have frequency-mixing effects.  For now, assume null values  
    
|      |Low|Mid|High|High+|  
|------|---|---|----|-----|  
|A1   |6.4|10|0|0|  
|A2   |6.4|5|0|0|    
|A3   |0|3|0|0|
|A4   |0|3|0|0|  
|A5    |0|1.5|0|0|
|A6    |0|1.5|0|0|  

In [6]:
2*np.pi/(550e3*np.sqrt(1e-10))

1.1423973285781064